
# **Swing Pattern Grid — v9 (Optuna Multi‑Objective)**  
Maximize **P&L**, minimize **Drawdown**, maximize **Sharpe** using Optuna.  
Uses **MOTPE** if available, otherwise falls back to **NSGAII** automatically.

Includes:
- Pattern scanner (bullish/bearish) + indicator confirmations (RSI, MACD, ADX, SMA cross, Bollinger squeeze, Volume filters).
- Groww-like costs/slippage, multiple stop styles (fixed %, ATR, prior candle, optional SMA-trail/Donchian).
- Multithreaded downloads & backtests, local caching, **live CSV logs**.
- Train (2005–2015) ⇒ pick Pareto params ⇒ OOS test (2016–2025).


In [7]:

# ===================================
# Imports & Global Configuration
# ===================================
import os, json, warnings, math, hashlib, time
from typing import Dict, Any, List, Optional, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore")

# ---------- Universe (sample) ----------
TICKERS = ["HDFCBANK.NS","RELIANCE.NS","TCS.NS","INFY.NS","ICICIBANK.NS"]

# ---------- Trade Direction ----------
TRADE_SIDE = "long"   # 'both' | 'long' | 'short'

# ---------- Dates ----------
TRAIN_START = "2005-01-01"
TRAIN_END   = "2015-12-31"
TEST_START  = "2016-01-01"
TEST_END    = "2025-09-26"  # None -> latest
WARMUP_DAYS = 400

# ---------- Trade & Exit Rules ----------
CAPITAL_PER_TRADE = 50000
MIN_CONFIRMATIONS_DEFAULT = 2
MAX_HOLD_DEFAULT = 10

STOP_RULES = dict(
    use_fixed_pct=True, fixed_pct=0.05,
    use_atr=True, atr_len=14, atr_mult=2.0,
    use_pattern=True, pattern_lookback=1,
    use_sma_trail=False, sma_trail_len=20,
    use_donchian=False, donchian_len=20,
)

# ---------- Costs & Slippage ----------
USE_COSTS = True
SLIPPAGE_PCT_PER_LEG = 0.0005
BROKERAGE_PER_LEG_RS = 0.0
STT_SELL_PCT = 0.001
EXCHANGE_TXN_PCT = 0.0000345
SEBI_PCT = 0.000001
GST_PCT_ON_BROKERAGE = 0.18
STAMP_DUTY_BUY_PCT = 0.00015

# ---------- Patterns ----------
PATTERNS_TO_USE = [
    "ENGULFING","PIERCING","MORNING_STAR","EVENING_STAR","HARAMI","HARAMI_CROSS",
    "HAMMER","INVERTED_HAMMER","SHOOTING_STAR","HANGING_MAN","DOJI","DARK_CLOUD_COVER",
]

# ---------- Indicator space ----------
PARAM_GRID = dict(
    USE_RSI   =[True, False],
    RSI_LEN   =[14, 21],
    RSI_LONG_MIN=[45, 50, 55],
    RSI_SHORT_MAX=[45, 50],
    USE_MACD  =[True, False],
    MACD_FAST =[12],
    MACD_SLOW =[26],
    MACD_SIGNAL=[9],
    MACD_MODE =["hist_above0","line_cross"],
    USE_ADX   =[True, False],
    ADX_LEN   =[14],
    ADX_MIN   =[16, 18, 20, 22],
    USE_SMA   =[True, False],
    SMA_FAST  =[5, 10, 20],
    SMA_SLOW  =[30, 50, 100],
    USE_BB    =[True, False],
    BB_LEN    =[20],
    BB_STD    =[2.0],
    BB_SQUEEZE_PCTL=[5, 10, 15],
    USE_VOLUME=[True, False],
    VOL_LOOKBACK=[10, 20, 50],
    VOL_MIN_AVG=[500_000, 1_000_000, 2_000_000],
    VOL_MIN_SURGE_RATIO=[1.2, 1.3, 1.5],
    MIN_CONFIRMATIONS=[1, 2, 3],
    MAX_HOLD=[5, 7, 10],
)

# ---------- Caching & Parallelism ----------
CACHE_DIR = "cache"
CACHE_OHLC_DIR = os.path.join(CACHE_DIR, "ohlc_range")
CACHE_INDICATOR_DIR = os.path.join(CACHE_DIR, "indicators")
os.makedirs(CACHE_OHLC_DIR, exist_ok=True)
os.makedirs(CACHE_INDICATOR_DIR, exist_ok=True)

CACHE_MAX_AGE_DAYS = 5
CACHE_INDICATORS = False

MAX_WORKERS_DOWNLOAD = 8
MAX_WORKERS_BACKTEST = 8

# ---------- Output ----------
OUT_ROOT = "outputs/grid_v9_optuna_moo"
os.makedirs(OUT_ROOT, exist_ok=True)

pd.set_option("display.max_rows", 180)
pd.set_option("display.width", 220)
print("Config loaded.")


Config loaded.


In [8]:

# ===================================
# Helpers (fixed _scalar_at scope)
# ===================================
def _coerce_ohlcv_1d(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for name in ["Open","High","Low","Close","Adj Close","Volume"]:
        if name in out.columns:
            obj = out[name]
            out[name] = obj.iloc[:, 0] if hasattr(obj, "ndim") and obj.ndim == 2 else obj
    return out

def _dedup_cols(df: pd.DataFrame) -> pd.DataFrame:
    if getattr(df.columns, "duplicated", None) is not None and df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()].copy()
    return df

def _get_series(df: pd.DataFrame, name: str):
    if name not in df.columns: return None
    obj = df[name]
    ser = obj.iloc[:,0] if isinstance(obj, pd.DataFrame) else obj
    return ser.reindex(df.index)

def _first_series(df: pd.DataFrame, name_options):
    for nm in name_options:
        if nm in df.columns:
            obj = df[nm]
            s = obj.iloc[:,0] if hasattr(obj,"ndim") and obj.ndim == 2 else obj
            return pd.to_numeric(s, errors="coerce")
    raise KeyError(f"None of {name_options} found")

def _as_bool_series(x, index):
    if isinstance(x, pd.Series):   return x.astype(bool).reindex(index)
    if isinstance(x, pd.DataFrame):return x.iloc[:,0].astype(bool).reindex(index)
    return pd.Series(x, index=index).astype(bool)

def _and_all(series_list, index):
    if not series_list: return pd.Series(True, index=index)
    out = _as_bool_series(series_list[0], index)
    for s in series_list[1:]: out = out & _as_bool_series(s, index)
    return out

def _scalar_at(df: pd.DataFrame, col: str, i: int) -> float:
    # No local import of pandas — use global pd to avoid UnboundLocalError in Python 3.11+
    obj = df[col]
    if isinstance(obj, pd.DataFrame):
        obj = obj.iloc[:, 0]
    val = obj.iloc[i]
    if isinstance(val, (float, int, np.floating, np.integer)):
        return float(val)
    if isinstance(val, (list, tuple, np.ndarray)):
        arr = np.asarray(val).ravel()
        return float(arr[0]) if arr.size else float('nan')
    try:
        return float(pd.to_numeric(pd.Series([val]), errors="coerce").iloc[0])
    except Exception:
        return float('nan')

print("Helpers ready.")


Helpers ready.


In [9]:

# ===================================
# Indicators & Patterns
# ===================================
def sma(series, n): return series.rolling(n, min_periods=n).mean()
def ema(series, n): return series.ewm(span=n, adjust=False).mean()

def rsi(series, n=14):
    s = pd.to_numeric(series.iloc[:,0], errors="coerce") if isinstance(series, pd.DataFrame) else pd.to_numeric(series, errors="coerce")
    delta = s.diff()
    gain = delta.clip(lower=0.0); loss = (-delta).clip(lower=0.0)
    roll_up = gain.rolling(n, min_periods=n).mean(); roll_down = loss.rolling(n, min_periods=n).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    return (100.0 - (100.0 / (1.0 + rs))).fillna(0.0)

def true_range(df: pd.DataFrame) -> pd.Series:
    H = _first_series(df, ["High","high"]); L = _first_series(df, ["Low","low"]); C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    prev_close = C.shift(1)
    return pd.concat([ (H-L).abs(), (H-prev_close).abs(), (L-prev_close).abs() ], axis=1).max(axis=1)

def atr(df: pd.DataFrame, n=14): return true_range(df).rolling(n, min_periods=n).mean()

def adx(df: pd.DataFrame, n=14):
    H = _first_series(df, ["High","high"]); L = _first_series(df, ["Low","low"]); C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    up_move   = H.diff(); down_move = -L.diff()
    plus_dm  = up_move.clip(lower=0.0); minus_dm = down_move.clip(lower=0.0)
    plus_dm  = plus_dm.where(plus_dm >= minus_dm, 0.0); minus_dm = minus_dm.where(minus_dm > plus_dm, 0.0)
    tr = true_range(pd.DataFrame({"High": H, "Low": L, "Close": C}))
    atr_v = tr.rolling(n, min_periods=n).mean()
    plus_di  = 100.0 * (plus_dm.rolling(n, min_periods=n).mean()  / atr_v)
    minus_di = 100.0 * (minus_dm.rolling(n, min_periods=n).mean() / atr_v)
    denom = (plus_di + minus_di).replace(0, np.nan)
    dx = ((plus_di - minus_di).abs() / denom) * 100.0
    adx_v = dx.rolling(n, min_periods=n).mean()
    return adx_v.fillna(0.0), plus_di.fillna(0.0), minus_di.fillna(0.0)

def pattern_flags(df: pd.DataFrame, patterns):
    O = _first_series(df, ["Open","open"]); H = _first_series(df, ["High","high"])
    L = _first_series(df, ["Low","low"]);   C = _first_series(df, ["Close","Adj Close","close","adjclose"])
    def is_bullish(c,o): return c > o
    def is_bearish(c,o): return c < o
    def body(c,o): return (c-o).abs()
    def range_(h,l): return (h-l).abs()
    rng = range_(H,L); bod = body(C,O)
    small_body = bod <= (rng * 0.3)
    long_lower_shadow = (O - L).abs().where(C>=O, (C - L).abs()) >= rng*0.5
    long_upper_shadow = (H - O).abs().where(C>=O, (H - C).abs()) >= rng*0.5
    flags = {}
    prev_O, prev_H, prev_L, prev_C = O.shift(1), H.shift(1), L.shift(1), C.shift(1)
    if "ENGULFING" in patterns:
        bull = (is_bullish(C,O) & is_bearish(prev_C, prev_O) & (C >= prev_O) & (O <= prev_C))
        bear = (is_bearish(C,O) & is_bullish(prev_C, prev_O) & (C <= prev_O) & (O >= prev_C))
        flags['ENGULFING_BULL'] = bull.fillna(False); flags['ENGULFING_BEAR'] = bear.fillna(False)
    if "PIERCING" in patterns or "DARK_CLOUD_COVER" in patterns:
        prev_mid = (prev_O + prev_C)/2
        flags['PIERCING_BULL'] = (is_bearish(prev_C, prev_O) & is_bullish(C,O) & (O < prev_L) & (C > prev_mid) & (C < prev_O)).fillna(False)
        flags['DARK_CLOUD_COVER_BEAR'] = (is_bullish(prev_C, prev_O) & is_bearish(C,O) & (O > prev_H) & (C < prev_mid) & (C > prev_C)).fillna(False)
    if "MORNING_STAR" in patterns or "EVENING_STAR" in patterns:
        p2_O, p2_H, p2_L, p2_C = O.shift(2), H.shift(2), L.shift(2), C.shift(2)
        flags['MORNING_STAR_BULL'] = (is_bearish(prev_C, prev_O) & (small_body.shift(1)) & (prev_L < p2_L) & is_bullish(C,O) & (C > pd.concat([p2_O,p2_C], axis=1).min(axis=1))).fillna(False)
        flags['EVENING_STAR_BEAR'] = (is_bullish(prev_C, prev_O) & (small_body.shift(1)) & (prev_H > p2_H) & is_bearish(C,O) & (C < pd.concat([p2_O,p2_C], axis=1).max(axis=1))).fillna(False)
    if "HARAMI" in patterns or "HARAMI_CROSS" in patterns:
        prev_min = pd.concat([prev_O, prev_C], axis=1).min(axis=1); prev_max = pd.concat([prev_O, prev_C], axis=1).max(axis=1)
        inside = (O >= prev_min) & (O <= prev_max) & (C >= prev_min) & (C <= prev_max)
        flags['HARAMI_BULL'] = (is_bullish(C,O) & is_bearish(prev_C, prev_O) & inside).fillna(False)
        flags['HARAMI_BEAR'] = (is_bearish(C,O) & is_bullish(prev_C, prev_O) & inside).fillna(False)
        doji = bod <= (rng * 0.1)
        flags['HARAMI_CROSS_BULL'] = (doji & is_bearish(prev_C, prev_O) & inside).fillna(False)
        flags['HARAMI_CROSS_BEAR'] = (doji & is_bullish(prev_C, prev_O) & inside).fillna(False)
    if "HAMMER" in patterns: flags['HAMMER_BULL'] = (long_lower_shadow & (~long_upper_shadow) & (small_body)).fillna(False)
    if "HANGING_MAN" in patterns: flags['HANGING_MAN_BEAR'] = flags['HAMMER_BULL']
    if "INVERTED_HAMMER" in patterns: flags['INVERTED_HAMMER_BULL'] = (long_upper_shadow & (~long_lower_shadow) & (small_body)).fillna(False)
    if "SHOOTING_STAR" in patterns: flags['SHOOTING_STAR_BEAR'] = flags['INVERTED_HAMMER_BULL']
    if "DOJI" in patterns: flags['DOJI_NEUTRAL'] = (bod <= (rng * 0.05)).fillna(False)
    return pd.DataFrame(flags, index=O.index).fillna(False)
print("Indicators ready.")


Indicators ready.


In [10]:

# ===================================
# Download, Precompute, Caching + CSV logger
# ===================================
def _append_csv(path: str, row: dict, field_order: list):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    is_new = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if is_new: f.write(",".join(field_order) + "\n")
        def _fmt(v):
            if isinstance(v, str): return '"' + v.replace('"','""') + '"'
            return "" if v is None else str(v)
        f.write(",".join(_fmt(row.get(k, "")) for k in field_order) + "\n")

def add_precomputed_indicators(df: pd.DataFrame, requirements: Dict[str, Any], stop_rules: Dict[str, Any]) -> pd.DataFrame:
    df = _dedup_cols(_coerce_ohlcv_1d(df.copy()))
    close = _first_series(df, ["Close","Adj Close","close","adjclose"])
    high  = _first_series(df, ["High","high"])
    low   = _first_series(df, ["Low","low"])
    try: vol = _first_series(df, ["Volume","volume"])
    except KeyError: vol = pd.Series(np.nan, index=df.index)
    df["Close"]=close; df["High"]=high; df["Low"]=low
    if "Volume" in df.columns or not vol.isna().all(): df["Volume"]=vol

    for n in sorted(requirements.get('RSI_LEN', set())): df[f'RSI_{n}'] = rsi(close, n)
    for (f, s) in sorted(requirements.get('MACD_FS', set())):
        ml = ema(close, f) - ema(close, s); df[f'MACD_LINE_{f}_{s}'] = ml
        for sig in sorted(requirements.get('MACD_SIGNAL', set())):
            ms = ema(ml, sig); df[f'MACD_SIGNAL_{f}_{s}_{sig}'] = ms; df[f'MACD_HIST_{f}_{s}_{sig}'] = ml - ms
    for n in sorted(requirements.get('ADX_LEN', set())):
        adx_v, plus_di, minus_di = adx(df, n); df[f'ADX_{n}']=adx_v; df[f'DI_PLUS_{n}']=plus_di; df[f'DI_MINUS_{n}']=minus_di
    for n in sorted(requirements.get('SMA', set())): df[f'SMA_{n}'] = close.rolling(n, min_periods=n).mean()
    for (n, k) in sorted(requirements.get('BB_NK', set())):
        ma = close.rolling(n, min_periods=n).mean(); sd = close.rolling(n, min_periods=n).std(ddof=0)
        up = ma + k*sd; lo = ma - k*sd; bw = (up - lo) / ma
        df[f'BB_MA_{n}_{k}']=ma; df[f'BB_UP_{n}_{k}']=up; df[f'BB_LO_{n}_{k}']=lo; df[f'BB_BW_{n}_{k}']=bw
    for n in sorted(requirements.get('VOL_LOOKBACK', set())):
        vma = vol.rolling(n, min_periods=n).mean(); df[f'VOL_MA_{n}']=vma; df[f'VOL_SURGE_{n}']=(vol/vma).replace([np.inf,-np.inf],np.nan)

    if stop_rules.get('use_atr', False):
        ln = int(stop_rules.get('atr_len', 14)); df[f'ATR_{ln}'] = true_range(pd.DataFrame({"High": high, "Low": low, "Close": close})).rolling(ln, min_periods=ln).mean()
    if stop_rules.get('use_sma_trail', False):
        ln = int(stop_rules.get('sma_trail_len', 20)); df[f'SMA_TRAIL_{ln}'] = sma(close, ln)
    if stop_rules.get('use_donchian', False):
        ln = int(stop_rules.get('donchian_len', 20)); df[f'DONCH_H_{ln}'] = high.rolling(ln, min_periods=ln).max(); df[f'DONCH_L_{ln}'] = low.rolling(ln, min_periods=ln).min()
    return _dedup_cols(df)

def collect_requirements(param_grid: Dict[str, List[Any]]) -> Dict[str, Any]:
    req = dict(RSI_LEN=set(), MACD_FS=set(), MACD_SIGNAL=set(), ADX_LEN=set(), SMA=set(), BB_NK=set(), VOL_LOOKBACK=set())
    if any(param_grid.get('USE_RSI', [False])): req['RSI_LEN'] = set(param_grid.get('RSI_LEN', []))
    if any(param_grid.get('USE_MACD', [False])): req['MACD_FS'] = set((f, s) for f in param_grid.get('MACD_FAST', []) for s in param_grid.get('MACD_SLOW', [])); req['MACD_SIGNAL'] = set(param_grid.get('MACD_SIGNAL', []))
    if any(param_grid.get('USE_ADX', [False])): req['ADX_LEN'] = set(param_grid.get('ADX_LEN', []))
    if any(param_grid.get('USE_SMA', [False])): req['SMA'] = set(param_grid.get('SMA_FAST', []) + param_grid.get('SMA_SLOW', []))
    if any(param_grid.get('USE_BB', [False])):  req['BB_NK'] = set((n, k) for n in param_grid.get('BB_LEN', []) for k in param_grid.get('BB_STD', []))
    if any(param_grid.get('USE_VOLUME', [False])): req['VOL_LOOKBACK'] = set(param_grid.get('VOL_LOOKBACK', []))
    return req

def cache_path_ohlc_range(ticker: str, start: str, end: Optional[str]) -> str:
    safe = ticker.replace("/", "_"); end_key = end or "None"
    return os.path.join("cache/ohlc_range", f"{safe}__{start}__{end_key}.pkl")

def is_fresh(path: str, max_age_days: int) -> bool:
    if not os.path.isfile(path): return False
    age_days = (time.time() - os.path.getmtime(path)) / 86400.0
    return age_days <= max_age_days

def download_ohlc_range(ticker: str, start: str, end: Optional[str]) -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)
    if df is None or df.empty: return pd.DataFrame()
    return df.dropna().copy()

def fetch_ohlc_range_with_cache(ticker: str, start: str, end: Optional[str]) -> pd.DataFrame:
    p = cache_path_ohlc_range(ticker, start, end); os.makedirs(os.path.dirname(p), exist_ok=True)
    if is_fresh(p, CACHE_MAX_AGE_DAYS):
        try: return pd.read_pickle(p)
        except Exception: pass
    df = download_ohlc_range(ticker, start, end)
    if not df.empty:
        try: df.to_pickle(p)
        except Exception: pass
    return df

def fetch_all_ohlc_range(tickers: List[str], start: str, end: Optional[str]) -> Dict[str, pd.DataFrame]:
    out = {}
    with ThreadPoolExecutor(max_workers=MAX_WORKERS_DOWNLOAD) as ex:
        futs = {ex.submit(fetch_ohlc_range_with_cache, t, start, end): t for t in tickers}
        for fut in as_completed(futs):
            t = futs[fut]
            try:
                df = fut.result()
                if not df.empty: out[t] = df
            except Exception as e:
                print("Download error:", t, e)
    return out

def fetch_enriched_frames_for_range(tickers: List[str], param_grid: Dict[str, List[Any]], start: str, end: Optional[str], warmup_days: int) -> Dict[str, pd.DataFrame]:
    start_ts = pd.Timestamp(start); fetch_start = (start_ts - pd.Timedelta(days=warmup_days)).strftime("%Y-%m-%d")
    raw = fetch_all_ohlc_range(tickers, fetch_start, end)
    req = collect_requirements(param_grid); enriched_map = {}
    for t, df in raw.items():
        if df.empty: continue
        enriched_map[t] = add_precomputed_indicators(df.copy(), req, STOP_RULES)
    return enriched_map

def slice_frames(frames: Dict[str, pd.DataFrame], start: str, end: Optional[str]) -> Dict[str, pd.DataFrame]:
    start_ts = pd.Timestamp(start); end_ts = pd.Timestamp(end) if end else None
    out = {}
    for t, df in frames.items():
        if df.empty: continue
        di = df.copy(); idx = di.index.normalize()
        mask = (idx >= start_ts.normalize())
        if end_ts is not None: mask &= (idx <= end_ts.normalize())
        out[t] = di.loc[mask].copy()
    return out

TRAIN_DIR = os.path.join(OUT_ROOT, f"train_{TRAIN_START}_{TRAIN_END}")
TEST_DIR  = os.path.join(OUT_ROOT, f"test_{TEST_START}_{TEST_END}")
os.makedirs(TRAIN_DIR, exist_ok=True); os.makedirs(TEST_DIR, exist_ok=True)

LIVE_METRICS_CSV = os.path.join(TRAIN_DIR, "train_optuna_moo_live_metrics.csv")
LIVE_TRADES_CSV  = os.path.join(TRAIN_DIR, "train_optuna_moo_live_trades.csv")
METRIC_FIELDS = ["ts","step","trial","pnl_total","dd_max","sharpe","trades","win_rate","avg_net","params_json"]
TRADE_FIELDS  = ["ts","step","trial","ticker","side","signal_dt","entry_dt","entry_time","exit_dt","exit_time","pattern_detected","exit_reason","entry_px","exit_px","qty","gross_pnl","fees","net_pnl","params_json"]

print("Precompute & logger ready.")


Precompute & logger ready.


In [11]:

# ===================================
# Confirmations, Costs, Stops, Backtest
# ===================================
def indicator_conditions_from_precomp(df: pd.DataFrame, p: dict, side: str) -> dict:
    conds = {}; idx = df.index
    if p.get('USE_RSI', False):
        rs = _get_series(df, f"RSI_{p['RSI_LEN']}"); 
        if rs is not None: conds['RSI'] = (rs >= p['RSI_LONG_MIN']) if side == 'long' else (rs <= p['RSI_SHORT_MAX'])
    if p.get('USE_MACD', False):
        f,s,sg = p['MACD_FAST'], p['MACD_SLOW'], p['MACD_SIGNAL']
        if p['MACD_MODE'] == 'hist_above0':
            h = _get_series(df, f"MACD_HIST_{f}_{s}_{sg}"); 
            if h is not None: conds['MACD'] = (h > 0.0) if side == 'long' else (h < 0.0)
        else:
            l = _get_series(df, f"MACD_LINE_{f}_{s}"); sline = _get_series(df, f"MACD_SIGNAL_{f}_{s}_{sg}")
            if l is not None and sline is not None: conds['MACD'] = (l > sline) if side == 'long' else (l < sline)
    if p.get('USE_ADX', False):
        a = _get_series(df, f"ADX_{p['ADX_LEN']}"); 
        if a is not None: conds['ADX'] = (a >= p['ADX_MIN'])
    if p.get('USE_SMA', False):
        sf, ss = p['SMA_FAST'], p['SMA_SLOW']
        if sf < ss:
            fs = _get_series(df, f"SMA_{sf}"); sl = _get_series(df, f"SMA_{ss}")
            if fs is not None and sl is not None: conds['SMA'] = (fs > sl) if side == 'long' else (fs < sl)
    if p.get('USE_BB', False):
        bn, bk = p['BB_LEN'], p['BB_STD']; bw = _get_series(df, f"BB_BW_{bn}_{bk}")
        if bw is not None:
            thresh = bw.rolling(252, min_periods=252).quantile(p['BB_SQUEEZE_PCTL']/100.0)
            conds['BB'] = (bw <= thresh)
    if p.get('USE_VOLUME', False) and ('Volume' in df.columns or any(c.startswith('VOL_') for c in df.columns)):
        ln = p['VOL_LOOKBACK']; ma = _get_series(df, f"VOL_MA_{ln}"); sur = _get_series(df, f"VOL_SURGE_{ln}")
        clauses = []
        if p.get('VOL_MIN_AVG') is not None and ma is not None: clauses.append(ma >= float(p['VOL_MIN_AVG']))
        if p.get('VOL_MIN_SURGE_RATIO') is not None and sur is not None: clauses.append(sur >= float(p['VOL_MIN_SURGE_RATIO']))
        if clauses: conds['VOL'] = _and_all(clauses, idx)
    for k, v in list(conds.items()): conds[k] = _as_bool_series(v, idx)
    return conds

def indicator_confirmations(df: pd.DataFrame, p: dict, side: str, min_conf: int) -> pd.Series:
    conds = indicator_conditions_from_precomp(df, p, side)
    if not conds: return pd.Series(True, index=df.index)
    stacked = pd.concat([_as_bool_series(v, df.index) for v in conds.values()], axis=1)
    return (stacked.sum(axis=1) >= min_conf).reindex(df.index).fillna(False)

def add_slippage(px: float, side_leg: str) -> float:
    if SLIPPAGE_PCT_PER_LEG <= 0: return px
    return px * (1.0 + SLIPPAGE_PCT_PER_LEG) if side_leg=='buy' else px * (1.0 - SLIPPAGE_PCT_PER_LEG)

def calc_costs(buy_val: float, sell_val: float) -> float:
    if not USE_COSTS: return 0.0
    brokerage_total = BROKERAGE_PER_LEG_RS * 2.0
    exch = EXCHANGE_TXN_PCT * (buy_val + sell_val); sebi = SEBI_PCT * (buy_val + sell_val)
    gst  = GST_PCT_ON_BROKERAGE * (BROKERAGE_PER_LEG_RS * 2.0); stt  = STT_SELL_PCT * sell_val
    stamp = STAMP_DUTY_BUY_PCT * buy_val
    return brokerage_total + exch + sebi + gst + stt + stamp

def compute_initial_stop_levels(df: pd.DataFrame, i_signal: int, side: str, entry_px: float) -> Dict[str, float]:
    levels = {}
    if STOP_RULES.get('use_fixed_pct', False):
        pct = float(STOP_RULES.get('fixed_pct', 0.05))
        levels['fixed'] = entry_px * (1.0 - pct) if side=='long' else entry_px * (1.0 + pct)
    if STOP_RULES.get('use_atr', False):
        ln = int(STOP_RULES.get('atr_len', 14)); mult = float(STOP_RULES.get('atr_mult', 2.0))
        col = f'ATR_{ln}'; atr_val = float(_get_series(df, col).iloc[i_signal]) if col in df else float(atr(df, ln).iloc[i_signal])
        levels['atr'] = entry_px - mult * atr_val if side=='long' else entry_px + mult * atr_val
    if STOP_RULES.get('use_pattern', False):
        lb = max(1, int(STOP_RULES.get('pattern_lookback', 1)))
        lo = df['Low'].iloc[max(0, i_signal - lb + 1):i_signal+1].min(); hi = df['High'].iloc[max(0, i_signal - lb + 1):i_signal+1].max()
        levels['pattern'] = float(lo) if side=='long' else float(hi)
    return levels

def check_trailing_stops(df: pd.DataFrame, j: int, side: str) -> List[Tuple[str, float]]:
    hits = []; H = float(_scalar_at(df, 'High', j)); L = float(_scalar_at(df, 'Low', j))
    if STOP_RULES.get('use_sma_trail', False):
        ln = int(STOP_RULES.get('sma_trail_len', 20)); col = f'SMA_TRAIL_{ln}'
        if col in df and not pd.isna(df[col].iloc[j]):
            s = float(df[col].iloc[j])
            if side=='long' and L <= s: hits.append((f'sma_trail_{ln}', s))
            if side=='short' and H >= s: hits.append((f'sma_trail_{ln}', s))
    if STOP_RULES.get('use_donchian', False):
        ln = int(STOP_RULES.get('donchian_len', 20)); lo_col = f'DONCH_L_{ln}'; hi_col = f'DONCH_H_{ln}'
        lo_ok = (lo_col in df) and (not pd.isna(df[lo_col].iloc[j])); hi_ok = (hi_col in df) and (not pd.isna(df[hi_col].iloc[j]))
        if side=='long' and lo_ok and L <= float(df[lo_col].iloc[j]): hits.append((f'donchian_L_{ln}', float(df[lo_col].iloc[j])))
        if side=='short' and hi_ok and H >= float(df[hi_col].iloc[j]): hits.append((f'donchian_H_{ln}', float(df[hi_col].iloc[j])))
    return hits

def run_backtest_for_symbol(df: pd.DataFrame, params: Dict[str, Any], patterns: List[str], trade_side: str, min_conf: int, max_hold: int):
    df = _dedup_cols(_coerce_ohlcv_1d(df)); pats = pattern_flags(df, patterns)
    bull_cols = [c for c in pats.columns if c.endswith('_BULL')]; bear_cols = [c for c in pats.columns if c.endswith('_BEAR')]
    long_pat  = pats[bull_cols].any(axis=1) if bull_cols else pd.Series(False, index=df.index)
    short_pat = pats[bear_cols].any(axis=1) if bear_cols else pd.Series(False, index=df.index)
    long_conf  = indicator_confirmations(df, params, 'long',  min_conf); short_conf = indicator_confirmations(df, params, 'short', min_conf)
    long_entry  = (long_pat  & long_conf).fillna(False); short_entry = (short_pat & short_conf).fillna(False)
    if trade_side == "long":  short_entry[:] = False
    if trade_side == "short": long_entry[:]  = False

    trades = []; i = 0; N = len(df.index); idx = df.index
    def _ts_full(ts): return pd.Timestamp(ts).strftime("%Y-%m-%d %H:%M:%S")
    def _ts_date(ts): return str(pd.Timestamp(ts).date())
    def _ts_time(ts): return pd.Timestamp(ts).strftime("%H:%M:%S")

    while i < N:
        side = 'long' if long_entry.iloc[i] else ('short' if short_entry.iloc[i] else None)
        if side is None: i += 1; continue
        entry_bar = i + 1
        if entry_bar >= N: break
        signal_ts = idx[i]; entry_ts = idx[entry_bar]
        entry_open = float(_scalar_at(df, 'Open', entry_bar))
        entry_px = add_slippage(entry_open, 'buy' if side=='long' else 'sell')
        qty = int(CAPITAL_PER_TRADE // entry_px)
        if qty <= 0: i += 1; continue
        init_levels = compute_initial_stop_levels(df, i, side, entry_px)
        static_stop = (min(init_levels.values()) if side=='long' else max(init_levels.values())) if init_levels else None

        pats_cols = [c for c in (bull_cols if side=='long' else bear_cols) if pats[c].iloc[i]]
        conds = indicator_conditions_from_precomp(df, params, side)
        conf_true = [name for name, ser in conds.items() if bool(ser.iloc[i])]

        
        snap_cols = []
        if params.get('USE_RSI', False):
            snap_cols.append(f"RSI_{int(params.get('RSI_LEN',14))}")
        if params.get('USE_MACD', False):
            f = int(params.get('MACD_FAST',12)); s=int(params.get('MACD_SLOW',26)); sg=int(params.get('MACD_SIGNAL',9))
            snap_cols += [f"MACD_LINE_{f}_{s}", f"MACD_SIGNAL_{f}_{s}_{sg}", f"MACD_HIST_{f}_{s}_{sg}"]
        if params.get('USE_ADX', False):
            snap_cols.append(f"ADX_{int(params.get('ADX_LEN',14))}")
        if params.get('USE_SMA', False):
            snap_cols += [f"SMA_{int(params.get('SMA_FAST',10))}", f"SMA_{int(params.get('SMA_SLOW',50))}"]
        if params.get('USE_BB', False):
            bn = int(params.get('BB_LEN',20)); bk=float(params.get('BB_STD',2.0)); snap_cols.append(f"BB_BW_{bn}_{bk}")
        if params.get('USE_VOLUME', False):
            ln = int(params.get('VOL_LOOKBACK',20)); snap_cols += [f"VOL_MA_{ln}", f"VOL_SURGE_{ln}"]
        if STOP_RULES.get('use_atr', False):
            ln = int(STOP_RULES.get('atr_len', 14)); snap_cols.append(f'ATR_{ln}')
        if STOP_RULES.get('use_sma_trail', False):
            ln = int(STOP_RULES.get('sma_trail_len', 20)); snap_cols.append(f'SMA_TRAIL_{ln}')

        snapshot = {}

        for sc in snap_cols:
            if sc in df.columns:
                try:
                    val = _scalar_at(df, sc, i)
                    if not (pd.isna(val) or np.isinf(val)): snapshot[sc] = float(val)
                except Exception: pass

        exit_bar = None; reason = None; j = entry_bar
        while j < N and (j - entry_bar) <= max_hold:
            H = float(_scalar_at(df, 'High', j)); L = float(_scalar_at(df, 'Low', j)); C = float(_scalar_at(df, 'Close', j))
            if static_stop is not None:
                if side=='long' and L <= static_stop: exit_bar, reason, exit_px = j, "SL_STATIC", static_stop; break
                if side=='short' and H >= static_stop: exit_bar, reason, exit_px = j, "SL_STATIC", static_stop; break
            hits = check_trailing_stops(df, j, side)
            if hits: tag, level = hits[0]; exit_bar, reason, exit_px = j, f"SL_{tag.upper()}", level; break
            if (j - entry_bar) >= max_hold: exit_bar, reason, exit_px = j, "MAX_HOLD", C; break
            j += 1

        if exit_bar is None: exit_bar, reason = N - 1, "FORCE_LAST"; exit_px = float(_scalar_at(df, 'Close', exit_bar))

        exit_ts = idx[exit_bar]
        buy_val  = entry_px * qty if side=='long' else exit_px * qty
        sell_val = exit_px * qty if side=='long' else entry_px * qty
        fees = calc_costs(buy_val, sell_val) if USE_COSTS else 0.0
        gross = (exit_px - entry_px) * qty if side=='long' else (entry_px - exit_px) * qty
        net   = gross - fees

        trades.append(dict(
            signal_dt=_ts_full(signal_ts),
            entry_dt=_ts_full(entry_ts), entry_time=_ts_time(entry_ts),
            exit_dt=_ts_full(exit_ts),   exit_time=_ts_time(exit_ts),
            entry_date=_ts_date(entry_ts), exit_date=_ts_date(exit_ts),
            pattern_detected=",".join(pats_cols), exit_reason=reason,
            side=side, entry_px=entry_px, exit_px=exit_px, qty=qty,
            static_stop=static_stop, gross_pnl=gross, fees=fees, net_pnl=net,
            reason=reason, signal_bar=_ts_date(signal_ts), patterns=",".join(pats_cols),
            indicator_conf_true=",".join(conf_true),
            indicator_values=json.dumps(snapshot, ensure_ascii=False),
            stop_levels=json.dumps({k: float(v) for k, v in (init_levels or {}).items()}, ensure_ascii=False),
        ))

        i = exit_bar + 1

    tdf = pd.DataFrame(trades)
    metrics = compute_metrics(tdf)
    return tdf, metrics

def compute_metrics(trades_df: pd.DataFrame) -> Dict[str, Any]:
    if trades_df is None or trades_df.empty:
        return dict(trades=0, win_rate=0.0, avg_net=0.0, total_net=0.0, max_dd=0.0, sharpe=0.0)
    pnl = trades_df['net_pnl'].values
    wins = (pnl > 0).sum()
    wr = wins / len(pnl) if len(pnl) else 0.0
    avg = float(np.mean(pnl)) if len(pnl) else 0.0
    total = float(np.sum(pnl))
    eq = np.cumsum(pnl); peak = np.maximum.accumulate(eq) if len(eq) else np.array([])
    dd = peak - eq if len(eq) else np.array([])
    max_dd = float(np.max(dd)) if len(dd) else 0.0
    sd = float(np.std(pnl)) if len(pnl) else 0.0
    sharpe = (avg / sd) if sd > 1e-9 else 0.0
    return dict(trades=int(len(pnl)), win_rate=wr, avg_net=avg, total_net=total, max_dd=max_dd, sharpe=sharpe)

def run_backtest_universe_for_params(ticker_to_df: Dict[str, pd.DataFrame], params: Dict[str, Any], trade_side: str):
    min_conf = int(params.get('MIN_CONFIRMATIONS', MIN_CONFIRMATIONS_DEFAULT)); max_hold = int(params.get('MAX_HOLD', MAX_HOLD_DEFAULT))
    all_trades = []
    def worker(t, df):
        tdf, _ = run_backtest_for_symbol(df, params, PATTERNS_TO_USE, trade_side, min_conf, max_hold)
        if not tdf.empty: tdf.insert(0, 'ticker', t)
        return tdf
    with ThreadPoolExecutor(max_workers=MAX_WORKERS_BACKTEST) as ex:
        futs = {ex.submit(worker, t, df): t for t, df in ticker_to_df.items()}
        for fut in as_completed(futs):
            try:
                out = fut.result()
                if out is not None and not out.empty: all_trades.append(out)
            except Exception as e:
                print("Backtest error:", futs[fut], e)
    base_cols = ['ticker','signal_dt','entry_dt','entry_time','exit_dt','exit_time','entry_date','exit_date','pattern_detected','exit_reason','side','entry_px','exit_px','qty','static_stop','gross_pnl','fees','net_pnl','reason','signal_bar','patterns','indicator_conf_true','indicator_values','stop_levels']
    trades_df = pd.concat(all_trades, axis=0, ignore_index=True) if all_trades else pd.DataFrame(columns=base_cols)
    metrics = compute_metrics(trades_df)
    return trades_df, metrics
print("Backtest core ready.")


Backtest core ready.


In [12]:

# ===================================
# Prepare Data Once
# ===================================
enriched_all = fetch_enriched_frames_for_range(TICKERS, PARAM_GRID, start=TRAIN_START, end=TEST_END, warmup_days=WARMUP_DAYS)
train_frames = slice_frames(enriched_all, TRAIN_START, TRAIN_END)
test_frames  = slice_frames(enriched_all, TEST_START, TEST_END)
print("Prepared frames: train bars ~", sum(len(df) for df in train_frames.values()), "test bars ~", sum(len(df) for df in test_frames.values()))


Prepared frames: train bars ~ 13565 test bars ~ 12030


In [13]:

# ===================================
# Optuna Multi-Objective (MOTPE → NSGAII fallback)
# ===================================
try:
    import optuna
except Exception as e:
    optuna = None
    print("⚠️ Optuna not found. Install with: pip install optuna")

OPTUNA_STEPS = [ (0.0, 0.25), (0.0, 0.60), (0.0, 1.00) ]

def _global_time_bounds(frames: dict):
    mins, maxs = [], []
    for df in frames.values():
        if df.empty: continue
        mins.append(df.index[0].normalize()); maxs.append(df.index[-1].normalize())
    if not mins: return None, None
    return min(mins), max(maxs)

def _slice_by_fraction(frames: dict, f0: float, f1: float):
    tmin, tmax = _global_time_bounds(frames)
    if tmin is None: return {}
    span = (tmax - tmin)
    a = tmin + timedelta(days=int(span.days * f0))
    b = tmin + timedelta(days=int(span.days * f1))
    return slice_frames(frames, a.strftime("%Y-%m-%d"), b.strftime("%Y-%m-%d"))

def optuna_objective(trial):
    if optuna is None:
        raise RuntimeError("Optuna not installed. Run: pip install optuna")

    # --- sample params ---
    p = {}
    p['USE_RSI'] = trial.suggest_categorical("USE_RSI", [True, False])
    p['RSI_LEN'] = trial.suggest_categorical("RSI_LEN", [14, 21])
    p['RSI_LONG_MIN'] = trial.suggest_categorical("RSI_LONG_MIN", [45, 50, 55])
    p['RSI_SHORT_MAX'] = trial.suggest_categorical("RSI_SHORT_MAX", [45, 50])

    p['USE_MACD'] = trial.suggest_categorical("USE_MACD", [True, False])
    p['MACD_FAST'] = 12
    p['MACD_SLOW'] = 26
    p['MACD_SIGNAL'] = 9
    p['MACD_MODE'] = trial.suggest_categorical("MACD_MODE", ["hist_above0","line_cross"])

    p['USE_ADX'] = trial.suggest_categorical("USE_ADX", [True, False])
    p['ADX_LEN'] = 14
    p['ADX_MIN'] = trial.suggest_categorical("ADX_MIN", [16,18,20,22])

    p['USE_SMA'] = trial.suggest_categorical("USE_SMA", [True, False])
    p['SMA_FAST'] = trial.suggest_categorical("SMA_FAST", [5,10,20])
    p['SMA_SLOW'] = trial.suggest_categorical("SMA_SLOW", [30,50,100])
    if p['USE_SMA'] and p['SMA_FAST'] >= p['SMA_SLOW']:
        raise optuna.TrialPruned()

    p['USE_BB'] = trial.suggest_categorical("USE_BB", [True, False])
    p['BB_LEN'] = 20
    p['BB_STD'] = 2.0
    p['BB_SQUEEZE_PCTL'] = trial.suggest_categorical("BB_SQUEEZE_PCTL", [5,10,15])

    p['USE_VOLUME'] = trial.suggest_categorical("USE_VOLUME", [True, False])
    p['VOL_LOOKBACK'] = trial.suggest_categorical("VOL_LOOKBACK", [10,20,50])
    p['VOL_MIN_AVG'] = trial.suggest_categorical("VOL_MIN_AVG", [500_000, 1_000_000, 2_000_000])
    p['VOL_MIN_SURGE_RATIO'] = trial.suggest_categorical("VOL_MIN_SURGE_RATIO", [1.2, 1.3, 1.5])

    p['MIN_CONFIRMATIONS'] = trial.suggest_categorical("MIN_CONFIRMATIONS", [1,2,3])
    p['MAX_HOLD'] = trial.suggest_categorical("MAX_HOLD", [5,7,10])

    last_metrics = None
    for step_idx, (f0, f1) in enumerate(OPTUNA_STEPS, start=1):
        sub_frames = _slice_by_fraction(train_frames, f0, f1)
        trades_df, metrics = run_backtest_universe_for_params(sub_frames, p, TRADE_SIDE)
        last_metrics = metrics

        ts_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        mrow = dict(ts=ts_now, step=step_idx, trial=trial.number,
                    pnl_total=round(metrics.get("total_net",0.0),2),
                    dd_max=round(metrics.get("max_dd",0.0),2),
                    sharpe=round(metrics.get("sharpe",0.0),4),
                    trades=int(metrics.get("trades",0)), win_rate=round(metrics.get("win_rate",0.0),4),
                    avg_net=round(metrics.get("avg_net",0.0),2),
                    params_json=json.dumps(p))
        _append_csv(LIVE_METRICS_CSV, mrow, METRIC_FIELDS)

        if trades_df is not None and not trades_df.empty:
            for _, r in trades_df.iterrows():
                trow = dict(ts=ts_now, step=step_idx, trial=trial.number, ticker=r.get("ticker",""), side=r.get("side",""),
                            signal_dt=r.get("signal_dt",""), entry_dt=r.get("entry_dt",""), entry_time=r.get("entry_time",""),
                            exit_dt=r.get("exit_dt",""), exit_time=r.get("exit_time",""),
                            pattern_detected=r.get("pattern_detected", r.get("patterns","")), exit_reason=r.get("exit_reason", r.get("reason","")),
                            entry_px=r.get("entry_px",""), exit_px=r.get("exit_px",""), qty=r.get("qty",""),
                            gross_pnl=r.get("gross_pnl",""), fees=r.get("fees",""), net_pnl=r.get("net_pnl",""),
                            params_json=json.dumps(p))
                _append_csv(LIVE_TRADES_CSV, trow, TRADE_FIELDS)

    return last_metrics["total_net"], last_metrics["max_dd"], last_metrics["sharpe"]

if optuna is None:
    raise RuntimeError("Optuna not installed. Run: pip install optuna")

try:
    from optuna.samplers import MOTPESampler
    sampler = MOTPESampler(seed=123)
    print("Using MOTPESampler.")
except Exception:
    sampler = optuna.samplers.NSGAIISampler(seed=123, population_size=64)
    print("MOTPESampler not available; falling back to NSGAIISampler. To enable MOTPE: pip install -U optuna")

study = optuna.create_study(directions=['maximize','minimize','maximize'], sampler=sampler)

N_TRIALS = 150
study.optimize(optuna_objective, n_trials=N_TRIALS, gc_after_trial=True)

pareto = study.best_trials
df_trials = study.trials_dataframe()
df_trials.to_csv(os.path.join(TRAIN_DIR, "train_optuna_moo_trial_history.csv"), index=False)

rows = []
for tr in pareto:
    vals = tr.values
    rows.append(dict(trial=tr.number, total_net=vals[0], max_dd=vals[1], sharpe=vals[2], params=json.dumps(tr.params)))
pd.DataFrame(rows).sort_values(["total_net","sharpe"], ascending=[False, False]).to_csv(
    os.path.join(TRAIN_DIR, "train_optuna_moo_pareto.csv"), index=False
)
print("Pareto size:", len(pareto))


[I 2025-09-28 12:02:39,956] A new study created in memory with name: no-name-471e25d8-9fb6-4eb0-94a9-f239b1ada10b


MOTPESampler not available; falling back to NSGAIISampler. To enable MOTPE: pip install -U optuna


[I 2025-09-28 12:02:40,759] Trial 0 finished with values: [125136.05868562416, 19790.78018112392, 0.08408407205026061] and parameters: {'USE_RSI': True, 'RSI_LEN': 21, 'RSI_LONG_MIN': 55, 'RSI_SHORT_MAX': 45, 'USE_MACD': True, 'MACD_MODE': 'hist_above0', 'USE_ADX': False, 'ADX_MIN': 16, 'USE_SMA': False, 'SMA_FAST': 5, 'SMA_SLOW': 30, 'USE_BB': False, 'BB_SQUEEZE_PCTL': 5, 'USE_VOLUME': False, 'VOL_LOOKBACK': 50, 'VOL_MIN_AVG': 1000000, 'VOL_MIN_SURGE_RATIO': 1.2, 'MIN_CONFIRMATIONS': 2, 'MAX_HOLD': 7}.
[I 2025-09-28 12:02:43,227] Trial 1 finished with values: [51355.76433334927, 37966.11399273451, 0.018880615149327044] and parameters: {'USE_RSI': True, 'RSI_LEN': 14, 'RSI_LONG_MIN': 45, 'RSI_SHORT_MAX': 50, 'USE_MACD': True, 'MACD_MODE': 'hist_above0', 'USE_ADX': False, 'ADX_MIN': 16, 'USE_SMA': False, 'SMA_FAST': 10, 'SMA_SLOW': 30, 'USE_BB': True, 'BB_SQUEEZE_PCTL': 10, 'USE_VOLUME': True, 'VOL_LOOKBACK': 20, 'VOL_MIN_AVG': 500000, 'VOL_MIN_SURGE_RATIO': 1.5, 'MIN_CONFIRMATIONS': 2,

Pareto size: 59


In [15]:

# ===================================
# Choose Compromise Params and OOS
# ===================================
def compromise_score(vs):
    total_net, max_dd, sharpe = vs
    return float(total_net) - 0.5*float(max_dd) + 1000.0*float(sharpe)

best_trial = max(study.best_trials, key=lambda tr: compromise_score(tr.values))
best_params = best_trial.params

with open(os.path.join(TRAIN_DIR, "train_optuna_moo_best_params.json"), "w") as f:
    json.dump(best_params, f, indent=2)

def backtest_oos(frames: Dict[str, pd.DataFrame], params: Dict[str, Any], out_dir: str, label: str = "oos"):
    trades_df, metrics = run_backtest_universe_for_params(frames, params, TRADE_SIDE)
    os.makedirs(out_dir, exist_ok=True)
    if not trades_df.empty: trades_df.to_csv(os.path.join(out_dir, f"{label}_trades.csv"), index=False)
    with open(os.path.join(out_dir, f"{label}_metrics.json"), "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    return trades_df, metrics

oos_trades, oos_metrics = backtest_oos(test_frames, best_params, out_dir=os.path.join(OUT_ROOT, f"test_{TEST_START}_{TEST_END}"), label="oos_optuna_moo")
print("Chosen params (compromise):", best_params)
print("OOS Metrics:", oos_metrics)
display(oos_trades.head())


Backtest error: TCS.NS 'MACD_FAST'
Backtest error: HDFCBANK.NS 'MACD_FAST'
Backtest error: INFY.NS 'MACD_FAST'
Backtest error: ICICIBANK.NS 'MACD_FAST'
Backtest error: RELIANCE.NS 'MACD_FAST'
Chosen params (compromise): {'USE_RSI': False, 'RSI_LEN': 14, 'RSI_LONG_MIN': 50, 'RSI_SHORT_MAX': 50, 'USE_MACD': True, 'MACD_MODE': 'line_cross', 'USE_ADX': False, 'ADX_MIN': 20, 'USE_SMA': True, 'SMA_FAST': 10, 'SMA_SLOW': 100, 'USE_BB': False, 'BB_SQUEEZE_PCTL': 15, 'USE_VOLUME': True, 'VOL_LOOKBACK': 20, 'VOL_MIN_AVG': 2000000, 'VOL_MIN_SURGE_RATIO': 1.2, 'MIN_CONFIRMATIONS': 1, 'MAX_HOLD': 10}
OOS Metrics: {'trades': 0, 'win_rate': 0.0, 'avg_net': 0.0, 'total_net': 0.0, 'max_dd': 0.0, 'sharpe': 0.0}


,ticker,signal_dt,entry_dt,entry_time,exit_dt,exit_time,entry_date,exit_date,pattern_detected,exit_reason,...,static_stop,gross_pnl,fees,net_pnl,reason,signal_bar,patterns,indicator_conf_true,indicator_values,stop_levels
